Importing Data From Kaggle

In [75]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [76]:
!kaggle datasets download -d team-ai/spam-text-message-classification

spam-text-message-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [77]:
import zipfile
zip_ref = zipfile.ZipFile('/content/SPAM text message 20170820 - Data.csv','r')
zip_ref.extractall('/content')
zip_ref.close()

In [78]:
import pandas as pd
df = pd.read_csv('/content/SPAM text message- Data.csv')

FileNotFoundError: ignored

In [ ]:
df.head()

In [ ]:
df['Message'][0]

**Text  Cleaning and Preprocessing**

In [ ]:
import re

In [79]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [80]:
df['Message'].shape

(5572,)

In [81]:
df.isna().sum()

Category    0
Message     0
dtype: int64

In [82]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemmatizer= WordNetLemmatizer()

In [83]:
corpus =[]
for i in range(len(df['Message'])):
  data = re.sub('[^a-zA-Z\s0-9]','',df['Message'][i])
  data = data.lower()
  data = data.split()
  data = [lemmatizer.lemmatize(words) for words in data if words not in (stopwords.words('english'))]
  data = ' '.join(data)
  corpus.append(data)

In [84]:
corpus[0]

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

Vectorizing text data for feeding it to model


In [85]:
from sklearn.feature_extraction.text import CountVectorizer
cv  = CountVectorizer(max_features = 250)

In [86]:
x = cv.fit_transform(corpus).toarray()

In [87]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [88]:
x.shape

(5572, 250)

In [89]:
y =df['Category']
y

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: Category, Length: 5572, dtype: object

In [90]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
y

array([0, 0, 1, ..., 0, 0, 0])

Splitting Data For Traning and Testing

In [91]:
from sklearn.model_selection import train_test_split

In [92]:
x_train , x_test ,y_train, y_test = train_test_split(x,y,test_size =0.2 , random_state =0)

In [93]:
x_train.shape

(4457, 250)

In [94]:
x_test.shape

(1115, 250)

**Making Deep Learning Model**

In [95]:
import tensorflow.keras as keras
from keras.layers import Dense , Flatten , Dropout , SimpleRNN ,LSTM
from keras.models import Sequential

Making Simple ANN Model

In [96]:
model = Sequential()

In [97]:
model.add(Dense(32 , activation ='relu' , input_dim =250))
model.add(Dropout(0.2))
model.add(Dense(1, activation ='sigmoid'))
model.compile(optimizer ='adam' , loss ='binary_crossentropy', metrics =['accuracy'])

In [98]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 32)                8032      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 8,065
Trainable params: 8,065
Non-trainable params: 0
_________________________________________________________________


In [99]:
model.fit(x_train,y_train , validation_data =(x_test,y_test),epochs =100)

Epoch 1/100
140/140 [==============================] - 2s 5ms/step - loss: 0.4663 - accuracy: 0.8696 - val_loss: 0.2985 - val_accuracy: 0.8888
Epoch 2/100
140/140 [==============================] - 1s 6ms/step - loss: 0.2143 - accuracy: 0.9421 - val_loss: 0.1509 - val_accuracy: 0.9659
Epoch 3/100
140/140 [==============================] - 1s 6ms/step - loss: 0.1305 - accuracy: 0.9666 - val_loss: 0.1081 - val_accuracy: 0.9695
Epoch 4/100
140/140 [==============================] - 1s 6ms/step - loss: 0.1035 - accuracy: 0.9688 - val_loss: 0.0937 - val_accuracy: 0.9731
Epoch 5/100
140/140 [==============================] - 1s 4ms/step - loss: 0.0918 - accuracy: 0.9724 - val_loss: 0.0875 - val_accuracy: 0.9758
Epoch 6/100
140/140 [==============================] - 1s 4ms/step - loss: 0.0819 - accuracy: 0.9762 - val_loss: 0.0851 - val_accuracy: 0.9731
Epoch 7/100
140/140 [==============================] - 1s 4ms/step - loss: 0.0770 - accuracy: 0.9762 - val_loss: 0.0822 - val_accuracy: 0.9776

In [100]:
x_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Making SimpleRNN Model

In [101]:
model_srnn = Sequential()
model_srnn.add(SimpleRNN(32 , activation ='relu' , input_shape =(250,1) , return_sequences =False))
model_srnn.add(Dense(1, activation ='sigmoid'))
model_srnn.compile(optimizer ='adam' , loss ='binary_crossentropy', metrics =['accuracy'])

In [104]:
model_srnn.fit(x_train,y_train,validation_data=(x_test,y_test) , epochs =5)

Epoch 1/5
140/140 [==============================] - 25s 179ms/step - loss: 0.3925 - accuracy: 0.8683 - val_loss: 0.4109 - val_accuracy: 0.8565
Epoch 2/5
140/140 [==============================] - 24s 173ms/step - loss: 0.3871 - accuracy: 0.8683 - val_loss: 0.4159 - val_accuracy: 0.8565
Epoch 3/5
140/140 [==============================] - 24s 172ms/step - loss: 0.3926 - accuracy: 0.8683 - val_loss: 0.4107 - val_accuracy: 0.8565
Epoch 4/5
140/140 [==============================] - 24s 169ms/step - loss: 0.3907 - accuracy: 0.8683 - val_loss: 0.4110 - val_accuracy: 0.8565
Epoch 5/5
140/140 [==============================] - 23s 164ms/step - loss: 0.3909 - accuracy: 0.8683 - val_loss: 0.4133 - val_accuracy: 0.8565


**Making Machine Learning Model**

In [105]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [106]:
model_nb = nb.fit(x_train,y_train)

In [109]:
y_pred = model_nb.predict(x_test)

In [110]:
y_pred

array([0, 1, 0, ..., 0, 0, 0])

In [111]:
from sklearn.metrics import classification_report

In [112]:
cr =classification_report(y_test , y_pred)

In [113]:
print(cr)

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       955
           1       0.92      0.89      0.90       160

    accuracy                           0.97      1115
   macro avg       0.95      0.94      0.94      1115
weighted avg       0.97      0.97      0.97      1115



In [114]:
from sklearn.metrics import accuracy_score

In [115]:
accuracy_score(y_pred , y_test)

0.9721973094170404

In [116]:
from sklearn.metrics import confusion_matrix

In [117]:
confusion_matrix(y_test , y_pred)

array([[942,  13],
       [ 18, 142]])

In [118]:
df.shape

(5572, 2)

Importing Pickle file of all 3 model

In [119]:
import pickle

In [120]:
with open('model.pkl','wb') as file:
  pickle.dump(model,file)

In [122]:
with open('model_srnn.pkl','wb') as file:
  pickle.dump(model_srnn,file)
with open('model_nb.pkl','wb') as file:
  pickle.dump(model_nb,file)